In [1]:
import argparse
import os
from glob import glob
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.bedrock import BedrockEmbeddings

In [2]:
CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [3]:
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama


In [15]:
def query_rag(prompt_base_template, context_text, query):
    prompt_template = ChatPromptTemplate.from_template(prompt_base_template)
    prompt = prompt_template.format(context=context_text, question=query)

    model = Ollama(model="mistral")
    response_text = model.invoke(prompt)

    return response_text

In [13]:
PROMPT_TEMPLATE = """
Answer the question based only on the following news context:

--- start of news article ---
{context}
--- end of news article ---

{question}
"""

In [17]:
files = glob('./news/*.txt')

for file in files:

    with open(file, 'r') as f:
        lines = [line for line in f]
        f.close()

    context_text = ''.join(lines)
    context_text
    
    query_text = """Based only on the news article provided above, 
    provide a list of all entities (people, companies, places...) mentioned on it, 
    together with the sentiment associated with those entitites based on the 
    text present in the article that is related to them (classified as -2, -1, 0, 1, 2. -2 is the most negative sentiment and +2 is the most positive sentiment)
    and then a summary of all of the news artcile 
    
    Your answer should be structed as json in the following structure:
    {
        "summary": a string containing the summary of the news article
        "entities": a list of all entities mentioned in the news article alongside the sentiment related to them. Example [['entity1', -1], ['entity2', 0], ['entity3, 2]]
    }
    """

    print(file)
    result = query_rag(PROMPT_TEMPLATE, context_text, query_text)
    print(result)


./news/What-the-teams-said-Friday-practice-at-the-2024-Emilia-Romanga-Grand-Prix-Formula-1-.txt
 {
    "summary": "Alex Albon and Logan Sargeant both suffered setbacks during Friday's free practice sessions at the Emilia-Romagna Grand Prix. Albon experienced an electrical issue in FP1 that took a long time to diagnose and fix, resulting in missed track time for him and the team. In FP2, Sargeant had trouble getting the tyres working on low fuel. Both drivers are confident they can improve ahead of qualifying, but Williams faced compromised running plans due to the issues. The team is focusing on fine-tuning the car for tomorrow. Ferrari led the way in both practice sessions.",
    
    "entities": [
        ["Alex Albon", 0],
        ["Logan Sargeant", 0],
        ["Williams", -1],
        ["FP1", 0],
        ["FP2", 0],
        ["Emilia-Romagna Grand Prix", 0],
        ["Ferrari", 1],
        ["Leclerc", 0],
        ["FP1 track conditions", 0],
        ["FP2 track conditions", 0],
   